In [2]:
import backtrader as bt
from datetime import datetime

In [24]:
class MovingAverageStrategy(bt.Strategy):

    # Moving average parameters
    params = (('pfast',20),('pslow',50),)
    
    def __init__(self):
        # get the data we have provided
        # self.close_data = self.data.close
        self.close_data = self.datas[0].close
        
        # this is where we create the indicators
        self.fast_sma = bt.indicators.MovingAverageSimple(self.close_data,
                                                         period=self.params.pfast)
        self.slow_sma = bt.indicators.MovingAverageSimple(self.close_data,
                                                         period=self.params.pslow)        
        

    def next(self):
        
        # This function will be called automatically after
        # a new data point (stock price) is available
        # print("%s - %s" % (self.data.datetime.date(0), self.close_data[0]))        
        
        # we have to check whether we have already opened a long position
        if not self.position:
            # we can open a long position if needed
            if self.fast_sma[0] > self.slow_sma[0] \
                    and self.fast_sma[-1] < self.slow_sma[-1]:
                print('BUY')
                self.buy()
                
        else:
            # check whether to close the long position
            if self.fast_sma[0] < self.slow_sma[0] \
                    and self.fast_sma[-1] > self.slow_sma[-1]:
                print('CLOSE')
                self.close()

In [25]:
if __name__ == "__main__":
    
    cerebro = bt.Cerebro()
    
    # add data to the core
    stock_data = bt.feeds.YahooFinanceData(dataname='MSFT',
                                    fromdate=datetime(2010, 1, 1),
                                    todate=datetime(2017, 1, 1))
    
    # we have to add the data to Cerebro
    cerebro.adddata(stock_data)
    cerebro.addstrategy(MovingAverageStrategy)
    
    # run the strategy
    cerebro.run()

BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
CLOSE
BUY
